In [1]:
import pandas as pd
import numpy as np
from joblib import dump, load
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy.random import multinomial
import random
from sklearn.linear_model import Lasso

In [2]:
sales_path: str = "../data/sales_history.csv"
promo_path: str = "../data/promo_history.xlsx"

promo = pd.read_excel(promo_path, index_col=0)
sales = pd.read_csv(sales_path, index_col=0)


promo = promo.drop(columns=0)
sales['sale_dt'] = pd.to_datetime(sales['sale_dt'])

/home/royalcat/projects/hacks/accenture-promo-hack/venv/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
#этот код ужасен, если у вас не как минимум 21:9 монитор можете не пытатся его читать
#обработка продаж
sales_sum = sales.drop(['posid'], axis=1).groupby(['skutertiaryid', 'sale_dt']).sum().reset_index()

sales_sum['week'] = sales_sum['sale_dt'].dt.isocalendar().week
sales_sum['year'] = sales_sum['sale_dt'].dt.isocalendar().year
sales_sum = sales_sum.drop('sale_dt', axis=1)


In [4]:
#обработка промо
promo_sum = promo.copy(deep=True)
promo_sum['start_year'] = promo_sum['start_dttm'].dt.isocalendar().year
promo_sum['start_week'] = promo_sum['start_dttm'].dt.isocalendar().week
promo_sum['end_year'] = promo_sum['end_dttm'].dt.isocalendar().year
promo_sum['end_week'] = promo_sum['end_dttm'].dt.isocalendar().week
promo_sum = promo_sum.drop(['start_dttm', 'end_dttm', 'promotypeid'], axis=1).sort_values('skutertiaryid')

prices = {}
for itemid in sales['skutertiaryid'].unique():
    item_sales = sales[sales['skutertiaryid'] == itemid]
    prices[itemid] = (item_sales['salerevenuerub']/item_sales['soldpieces']).replace([np.inf, -np.inf], np.nan).max()

promo_revenue = []
promo_soldpieces = []

for i, row in promo_sum.iterrows():
    promo_sales = sales_sum[(sales_sum['week'] >= row['start_week']) & (sales_sum['week'] <= row['end_week']) & ((sales_sum['year'] == row['start_year']) | (sales_sum['year'] == row['end_year']))]
    promo_revenue.append(prices[row['skutertiaryid']]  * row['chaindiscountvalue'] * (promo_sales['soldpieces'].sum()))
    promo_soldpieces.append(promo_sales['soldpieces'].sum())

promo_sum['revenue'] = promo_revenue
promo_sum['soldpieces'] = promo_soldpieces
promo_sum = promo_sum[promo_sum['soldpieces'] != 0]

In [5]:
normal_sold = {} #нужно в глобал
for itemid in sales_sum['skutertiaryid'].unique():
    normal_sold[itemid] = sales_sum[sales_sum['skutertiaryid'] == itemid]['salerevenuerub'].median()

sold_added = []
for i, row in promo_sum.iterrows():
    sold_added.append((row['soldpieces']/normal_sold[row['skutertiaryid']]))
promo_sum['sold_added'] = sold_added


sold_added_med = {} #нужно в глобал
soldpieces_med = {} #нужно в глобал
for itemid in promo_sum['skutertiaryid'].unique():
    sold_added_med[itemid] = promo_sum[promo_sum['skutertiaryid'] == itemid]['sold_added'].median()
    soldpieces_med[itemid] = promo_sum[promo_sum['skutertiaryid'] == itemid]['soldpieces'].median()

In [6]:
promo_sum = promo_sum.drop(['end_year', 'end_week'], axis=1)

In [7]:
train_data = []
for itemid in promo_sum['skutertiaryid'].unique():
    promo_item = promo_sum[promo_sum['skutertiaryid'] == itemid].sort_values('start_week')
    last_week: int = 0
    for index, row in promo_item.iterrows():
        if row['start_week'] - last_week > 1:
            for i in range(last_week+1, int(row['start_week'])):
                week_sales = sales_sum[(sales_sum['year'] == row['start_year']) & (sales_sum['week'] == i) & (sales_sum['skutertiaryid'] == row['skutertiaryid'])]
                if len(week_sales) > 0:
                    train_data.append([
                                        int(row['skutertiaryid']),
                                        0,
                                        int(row['start_year']), 
                                        int(i), 
                                        0,
                                        week_sales.iloc[0]['soldpieces'],
                                        week_sales.iloc[0]['soldpieces']/normal_sold[row['skutertiaryid']]
                                       ])
                    last_week = int(row['start_week'])
                else:
                    train_data.append([int(row['skutertiaryid']), 0, int(row['start_year']), int(i), 0,\
                                    np.nan,\
                                    np.nan])
        train_data.append(row.values)
train_data = pd.DataFrame(train_data, columns=promo_sum.columns)

In [8]:
for i, row in train_data.iterrows():
    if np.isnan(row['soldpieces']):
        row['soldpieces'] = soldpieces_med[row['skutertiaryid']]
    if np.isnan(row['sold_added']):
        row['sold_added'] = sold_added_med[row['skutertiaryid']]

In [9]:
train_data = train_data.dropna()
weeks = train_data.sort_values('start_week')['start_week'].unique() #нужно в глобал
items = train_data['skutertiaryid'].unique() #нужно в глобал


In [16]:
lenc = LabelEncoder() # нужно в глобал

y = list(train_data['sold_added'])
X = list(zip(train_data['chaindiscountvalue'], train_data['soldpieces'], train_data['revenue'], train_data['start_week'], lenc.fit_transform(train_data['skutertiaryid'])))


reg = Lasso(alpha=0.1, positive=True) #нужно в глобал
reg.fit(X, y)

Lasso(alpha=0.1, positive=True)

In [17]:
soldpieces_median = {} #нужно в глобал
for itemid in items:
    sales_item = sales_sum[sales_sum['skutertiaryid'] == itemid]
    soldpieces_med[itemid] = {}
    for week in weeks:
        soldpieces_med[itemid][week] = sales_item[sales_item['week'] == week]['soldpieces'].median()

max_sale = {} # нужно в глобал
for itemid in items:
    max_sale[itemid] = promo_sum[promo_sum['skutertiaryid'] == itemid]['chaindiscountvalue'].max()

In [18]:
def get_random_distribution(n, all_sum, s=random.random()): #нужно в глобал
    return np.random.multinomial(all_sum, np.random.dirichlet(np.ones(n)*s))

In [19]:
def predict(itemid, next_revenue, seed = random.random(), tries=10):
    test_data = []
    for week in weeks:
        test_data.append([random.uniform(0, max_sale[itemid]), soldpieces_med[itemid][week], week, itemid])
    test_data = pd.DataFrame(test_data, 
                             columns=['chaindiscountvalue', 'soldpieces', 'start_week', 'skutertiaryid'])
    
    
    tries_dict = {}
    for i in range(tries):
        test_data['revenue'] = get_random_distribution(len(weeks), next_revenue, seed)
        X = list(zip(test_data['chaindiscountvalue'], test_data['soldpieces'],
                 test_data['revenue'], test_data['start_week'], lenc.transform(test_data['skutertiaryid'])))
        predict_result = reg.predict(X)
        tries_dict[np.sum(predict_result)] = predict_result
    
    test_data['sold_added'] = tries_dict[np.max(list(tries_dict.keys()))]
    test_data['soldpieces'] = (test_data['soldpieces']  * test_data['sold_added']).astype(int)

    return test_data

    


In [20]:
predict(7182, pow(10,10), seed=0.01, tries=1000)

,chaindiscountvalue,soldpieces,start_week,skutertiaryid,revenue,sold_added
0,0.169852,10974,1,7182,0,0.075980
1,0.114397,4389,2,7182,0,0.055460
2,0.118051,2243,3,7182,0,0.046645
3,0.042886,2240,4,7182,0,0.047959
4,0.166399,1012,5,7182,0,0.042019
5,0.087623,3488,6,7182,0,0.056622
6,0.042063,5251,7,7182,0,0.064973
7,0.011287,3956,8,7182,41,0.061224
8,0.030916,1054,9,7182,0,0.048382
9,0.160437,6592,10,7182,0,0.073405
